## Regresión Lineal (manual con pytorch):

Iniciamos con la carga de las librerias que utilizaremos:

In [1]:
import numpy as np
import torch

Analizaremos antes algunos aspectos de **pytorch**, primeramente como se crean tensores:

In [2]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [3]:
# Print tensors
print(x)
print(w)
print(b)

tensor(3.)
tensor(4., requires_grad=True)
tensor(5., requires_grad=True)


Nótese como algunas variables poseen la etiqueta `requires_grad=True` esto significa como se verá mas adelante que es posible calcular directamente las derivadas parciales de las variables que tengan este parámetro en `True`

Con los tensores creados anteriormente, podemos hacer operaciones aritméticas, en este caso construimos la ecuación de una linea recta e imprimimos el resultado:

In [4]:
y = w * x + b
print(y)

tensor(17., grad_fn=<AddBackward0>)


Ahora, calculamos las derivadas parciales de ***y*** respecto a las variables con `requires_grad=True` estas son `w` y `b`:

In [5]:
# Compute gradients
y.backward()

In [6]:
# Display gradients
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dw: tensor(3.)
dy/db: tensor(1.)


Con el panorama general de uso de ***pytorch*** para los propósitos de este ejercicio, ahora procedemos a analizar rapidamente los datos de nuestro problema:

Region | Temp (F) | Rain (mm) | Hum (%) | Apples (ton) | Oranges (ton)
------------ | ------------- | ------------- | ------------- | ------------- | -------------
Kanto | 73 | 67 | 43 | 56 | 70
Johto | 91 | 88 | 64 | 81 | 101
Hoenn | 87 | 134 | 58 | 119 | 133
Sinnoh | 102 | 43 | 37 | 22 | 37
Unova | 69 | 96 | 70 | 103 | 119


La idea es crear un modelo de regresión lineal de la forma:
    
    yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
    yeild_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2


$$
\begin{array} { l l l l l } {X} & \times & {W^{T}} & {+} & {b} \end{array}
$$

$$
\begin{bmatrix} {73} & {67} & {43} \\ {91} & {88} & {64} \\ {\vdots} & {\vdots} & {\vdots} \\ {69} & {96} & {70} \end{bmatrix} \times \begin{bmatrix}{w_{11}} & {w_{21}} \\ {w_{12}} & {w_{22}} \\ {w_{13}} & {w_{23}} \end{bmatrix} + \begin{bmatrix} {b_{1}} & {b_{2}} \\ {b_{1}} & {b_{2}} \\ {\vdots} & {\vdots} \\ {b_{1}} & {b_{2}} \end{bmatrix}
$$


Creamos ahora los arrays:

In [7]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [8]:
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [9]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [10]:
targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

Convertimos ahora los `inputs` y los `targets` a tensores:

In [11]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Todo modelo de aprendizaje inicia con un conjunto de pesos random, en este caso creamos los tensores `w` y `b` random:

In [12]:
torch.manual_seed(0)
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986]], requires_grad=True)
tensor([0.4033, 0.8380], requires_grad=True)


A continuación definimos nuestro modelo, que correspondería a la ecuación de recta de la regresión lineal (no olvidar que el operador `@` sirve para multiplicar dos matrices o bien tensores tipo 2):

In [13]:
# Define the model
def model(x):
    return x @ w.t() + b

Ahora con los pesos aleatorios procedemos a hacer la predicción a partir del modelo:

In [14]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  -0.4516,  -90.4691],
        [ -24.6303, -132.3828],
        [ -31.2192, -176.1530],
        [  64.3523,  -39.5645],
        [ -73.9524, -161.9561]], grad_fn=<AddBackward0>)


Comparamos con los datos originales:

In [15]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


A simple vista se nota como los valores obtenidos en el modelo son bastante lejanos a los esperados, esto es normal, ya que no hemos usado ninguna técnica de optimización.

Ahora tratemos de medir el error, en este caso con el ***MSE***:

In [16]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [17]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(33060.8047, grad_fn=<DivBackward0>)


Nótese como el resultado anterior obtenido es un valor alto, se espera que este valor sea lo más bajo posible:

Una tecníca de ***OPTIMIZACIóN*** muy común cosiste en el uso del gradiante descendiente, que corresponde a la derivada del error (en este caso el `loss` respecto a `w` y `b`:

In [18]:
# Compute gradients
loss.backward()

In [19]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986]], requires_grad=True)
tensor([[ -6938.4355,  -9674.6758,  -5744.0205],
        [-17408.7871, -20595.9336, -12453.4707]])


In [20]:
# Gradients for bias
print(b)
print(b.grad)

tensor([0.4033, 0.8380], requires_grad=True)
tensor([ -89.3802, -212.1051])


Dado que pytorch acumula los gradientes, debemos resetearlos a cero antes de seguir adelante:

In [21]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Procedemos ahora a contruir nuestro modelo con los ajustes al `w` y `b` usando el gradiante descendiente:

Primeramente generamos la primera predicción:
    

In [22]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  -0.4516,  -90.4691],
        [ -24.6303, -132.3828],
        [ -31.2192, -176.1530],
        [  64.3523,  -39.5645],
        [ -73.9524, -161.9561]], grad_fn=<AddBackward0>)


Ahora calculamos el `loss`:

In [23]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(33060.8047, grad_fn=<DivBackward0>)


Ahora calculamos los gradientes para `loss` respecto a `w` y `b`:

In [24]:
# Compute gradients
loss.backward()

In [25]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [26]:
print(w)

tensor([[ 1.6104, -0.1967, -2.1213],
        [ 0.7425, -0.8786, -1.2741]], requires_grad=True)


Ahora calculamos de nuevo el `loss`, debería ser menor que el anterior:

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(23432.4902, grad_fn=<DivBackward0>)


Efectivamente es menor, sin embargo vemos que los resultados obtenidos difieren considerablemente de los esperados:

In [28]:
preds

tensor([[  13.5663,  -58.6043],
        [  -6.1255,  -90.4440],
        [  -8.8863, -126.1837],
        [  77.7158,   -8.3414],
        [ -55.8554, -121.4523]], grad_fn=<AddBackward0>)

In [29]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Q/ Cómo se resuelve el problema de la optimización efectiva? 

R/ Se resuelve repitiendo el proceso múltiples veces, cada iteración se conoce como epoch:

In [34]:
# Train for 1000 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [35]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(11.0205, grad_fn=<DivBackward0>)


In [36]:
preds

tensor([[ 57.5609,  70.6524],
        [ 79.4107,  98.8859],
        [124.4177, 136.4671],
        [ 22.6800,  37.9871],
        [ 96.2028, 115.6331]], grad_fn=<AddBackward0>)

In [37]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Notamos ahora como los valores son más cercanos a los deseados.